In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

In [2]:
np.random.seed(42)

# 1. Data processing and Model Defination 

In [3]:
X_train = pd.read_csv("train_X_cat.csv",index_col=0)
Y_train = pd.read_csv("train_y_sales.csv",index_col=0)

In [4]:
Model_1 = BaggingRegressor()

# 2. Data Pipelining with GridSearch 

In [7]:

RG = Model_1
pipe = Pipeline(steps=[("RG",RG)])

param_grid = {
    "RG__bootstrap":[True,False],
    "RG__bootstrap_features":[True,False],
    "RG__n_estimators":[10,100,1000]
}
score_metric = 'neg_mean_squared_error'
search = GridSearchCV(pipe,param_grid,
                      n_jobs=-1,
                      cv=10,
                      scoring=score_metric)
search.fit(X_train,Y_train)

D:\Download\Anaconda\lib\site-packages\sklearn\ensemble\_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


GridSearchCV(cv=10, estimator=Pipeline(steps=[('RG', BaggingRegressor())]),
             n_jobs=-1,
             param_grid={'RG__bootstrap': [True, False],
                         'RG__bootstrap_features': [True, False],
                         'RG__n_estimators': [10, 100, 1000]},
             scoring='neg_mean_squared_error')

In [8]:
df = pd.DataFrame(search.cv_results_)
df = df.sort_values(by=['rank_test_score'])
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_RG__bootstrap,param_RG__bootstrap_features,param_RG__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
5,22.666686,1.732575,0.153800,0.028257,True,False,1000,"{'RG__bootstrap': True, 'RG__bootstrap_feature...",-677.683823,-318.038771,...,-339.147434,-1411.194910,-1623.992539,-963.656331,-2303.281142,-519.811484,-443.743782,-919.181534,624.959634,1
4,2.446502,0.322503,0.020197,0.008307,True,False,100,"{'RG__bootstrap': True, 'RG__bootstrap_feature...",-747.862558,-340.425445,...,-413.464199,-1636.489635,-1679.001732,-919.166941,-2283.723947,-461.733038,-469.084762,-952.440492,639.859794,2
2,23.870451,2.473302,0.149962,0.031693,True,True,1000,"{'RG__bootstrap': True, 'RG__bootstrap_feature...",-1207.395700,-513.909973,...,-411.541604,-2132.899749,-2100.154737,-630.765810,-1162.279622,-533.740444,-797.896903,-1037.405455,594.969059,3
1,2.323702,0.261727,0.017912,0.002749,True,True,100,"{'RG__bootstrap': True, 'RG__bootstrap_feature...",-1157.603103,-486.075667,...,-422.918678,-2015.526899,-2198.703351,-584.808220,-1281.449791,-493.111169,-829.496239,-1037.866630,601.807070,4
3,0.325032,0.076460,0.005134,0.001602,True,False,10,"{'RG__bootstrap': True, 'RG__bootstrap_feature...",-630.152658,-420.182089,...,-664.228608,-1673.742342,-1797.430759,-927.752420,-2959.750701,-473.850573,-308.544395,-1061.634739,793.448132,5
8,39.996148,3.491650,0.142995,0.007304,False,True,1000,"{'RG__bootstrap': False, 'RG__bootstrap_featur...",-1393.120005,-539.171412,...,-437.604557,-2206.033615,-1892.701928,-693.735960,-1846.351633,-641.780500,-861.798695,-1141.500800,608.391237,6
7,3.945677,0.347792,0.020067,0.005842,False,True,100,"{'RG__bootstrap': False, 'RG__bootstrap_featur...",-1405.640169,-495.853151,...,-548.558298,-2160.654680,-1756.910211,-773.667525,-2157.533952,-718.932296,-900.877245,-1184.294463,607.335079,7
0,0.239500,0.016035,0.005160,0.000712,True,True,10,"{'RG__bootstrap': True, 'RG__bootstrap_feature...",-1024.018924,-865.205190,...,-901.949114,-2895.569494,-1828.267722,-683.260828,-1441.728344,-585.770701,-970.110637,-1186.101779,673.891177,8
6,0.454286,0.065760,0.006260,0.004009,False,True,10,"{'RG__bootstrap': False, 'RG__bootstrap_featur...",-1263.493797,-476.164430,...,-597.554177,-2122.930127,-1609.945253,-770.999172,-2554.456242,-586.661720,-702.626178,-1211.659053,678.621881,9
11,34.456693,1.687933,0.110400,0.021114,False,False,1000,"{'RG__bootstrap': False, 'RG__bootstrap_featur...",-1598.398333,-514.340237,...,-977.717552,-1758.751004,-1731.404439,-1416.560555,-4006.780016,-1810.878340,-567.991346,-1520.447855,953.239139,10


# 3. Best Hyperparameter

In [9]:
parms = pd.DataFrame(df['params'])
for i in parms.iloc[0]:
    print(i)

{'RG__bootstrap': True, 'RG__bootstrap_features': False, 'RG__n_estimators': 1000}


# 4. Evaluation of the Model with K-Fold for Original Data


In [10]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error as MSE

# Assuming parms is a DataFrame containing the best parameters
best_boot = parms.iloc[0][0]['RG__bootstrap']
best_boot_feat = parms.iloc[0][0]['RG__bootstrap_features']
best_n_est = parms.iloc[0][0]['RG__n_estimators']

# Initialize the BaggingRegressor with extracted parameters
Model_2 = BaggingRegressor(bootstrap=best_boot, bootstrap_features=best_boot_feat, n_estimators=best_n_est)

# Prepare the k-fold cross-validation configuration
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=132)

# List to store the MSE for each fold
mse_values = []

for train_index, test_index in kf.split(X_train):
    # Split the data into training and validation folds
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    Y_train_fold, Y_val_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
    # Fit the model on the training fold
    Model_2.fit(X_train_fold, Y_train_fold.values.ravel())
    
    # Predict on the validation fold and calculate the MSE
    Y_val_fold_pred = Model_2.predict(X_val_fold)
    mse_fold = MSE(Y_val_fold, Y_val_fold_pred)
    mse_values.append(mse_fold)
   
# Calculate the mean MSE across all folds
mean_mse = np.mean(mse_values)
print(f'Mean MSE across all folds: {mean_mse}')


Mean MSE across all folds: 1187.7391974524312


# 5. Evaluation of the Model with K-Fold for Log(Sales) Data



In [13]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE

# Assuming parms is a DataFrame containing the best parameters
best_boot = parms.iloc[0][0]['RG__bootstrap']
best_boot_feat = parms.iloc[0][0]['RG__bootstrap_features']
best_n_est = parms.iloc[0][0]['RG__n_estimators']

# Initialize the BaggingRegressor with extracted parameters
Model_3 = BaggingRegressor(bootstrap=best_boot, bootstrap_features=best_boot_feat, n_estimators=best_n_est)

# Load the target variable, which is the log-transformed sales, from a CSV file
Y_train = pd.read_csv("train_y_logsales.csv", index_col=0)
# Assuming X_train is already defined and holds the feature data

# Prepare the k-fold cross-validation configuration
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=132)

# List to store the MSE for each fold
mse_values = []

for train_index, test_index in kf.split(X_train):
    # Split the data into training and validation folds
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    Y_train_fold, Y_val_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
    # Fit the model on the training fold
    Model_3.fit(X_train_fold, Y_train_fold.values.ravel())
    
    # Predict on the validation fold and calculate the MSE
    Y_val_fold_pred = Model_3.predict(X_val_fold)
    mse_fold = MSE(np.exp(Y_val_fold), np.exp(Y_val_fold_pred))
    mse_values.append(mse_fold)
   
# Calculate the mean MSE across all folds
mean_mse = np.mean(mse_values)
print(f'Mean MSE across all folds (original sales scale): {mean_mse}')

Mean MSE across all folds (original sales scale): 1159.9940920920387
